# Melna ķermeņa starojums #
Šajā klādē rēķināsim melna ķermeņa starojumu. 

## Mācību mērķi ##
Pēc šīs aktivitātes, studenti varēs:
1. Skaitliski rēķināt melna ķermeņa emisijas spektru
2. Skaitliski integrēt melna ķermeņa emisijas spektru pa viļņa garumu, lai iegūtu emisijas spēju jeb enerģētisko spīdību
3. Empiriski iegūt Vīna pārbīdes likumu par viļņa garumu, pie kura melnais ķermenis izstaro maksimālo intensitāti
4. Saistīt zvaigžņu krāsas ar to temperatūru
5. Empiriski iegūt Stefāna-Bolcmaņa likumu par absolūti melna ķermeņa integrālo emisijas spēju

## Teorija ##
Maks Planks uzrakstīja formulu, lai rēķinātu absolūti melna ķermeņa emisijas spektru, t.i., spektrālā intensitāte, ko izstaro absolūti melnais ķermenis ar temperatūru $T$ pie viļņa garuma $\lambda$.
$$
B_{\lambda}(\lambda,T)=2\frac{hc^2}{\lambda^5}\frac{1}{e^{\frac{hc}{\lambda k_B T}}-1}
$$
Mērvienības ir $\mathrm{W}\mathrm{m}^{-2}\mathrm{sr}^{-1}$. Tas nozīmē, ja mēs gribam rēķināt enerģiju, ko izstaro absolūti melnais ķermenis, mums ir jāreizina ar melnā ķermeņa virsmas laukumu un atveres leņķi uz kuru staro. 

## Dažas tehniskas piezīmes ##
### Julia valoda ###
Mēs izmantosim [_Julia_](https://julialang.org/) valodu. _Julia_ valoda ir salīdzinoši vienkārša, līdzīgi kā _python_, iespējams vēl vienkāršāk izmantot. Tomēr, _Julia_ rēķina gandrīz tik pat ātri kā _C/C++_. 


### Julia instalešana ###
Pirms sākt rēķināt, mums ir jāizmanto dažas specializētas programmatūras paketes. Tie ir jābūt jau installētiem mūsu Julia vidē izmantojot paketes menedžeri jeb komandas:

`using Pkg 
Pkg.add("myPackage")
`

Šim uzdevumam izmantosim šāda paketes:
1. [Interact.jl](https://www.juliapackages.com/p/interact) Šī pakete ļaujs interaktīvi manipulēt grafiskas vides
2. [QuadGK.jl](https://www.juliapackages.com/p/quadgk)    Šī pakete ļaujs skaitliski integrēt kādu funkciju
3. [Unitful.jl](https://www.juliapackages.com/p/unitful)   Šī pakete ļaujs definēt mērvienības mūsu mainīgajiem
4. [Plots.jl](https://www.juliapackages.com/p/plots)     Šī pakete ļaujs veidot grafikus

Turklāt izmantosim plotly() funkciju, kas ļauj interaktīvi strādāt ar grafikiem. 

In [1]:
using Interact   # lai veidotu manipulatorus
using QuadGK     # lai veiktu skaitlisko integrēšanu
using Unitful    # lai pievienotu mērvienības
using Plots      # lai zīmētu grafikus
using UnitfulRecipes
plotly()         # PlotlyBase lai pievienotu funkcionalitāti grāfikām (piem. interaktivitāte)

WebIO._IJuliaInit()

Plots.PlotlyBackend()

## Konstantes ##
Tagad definēsim konstantes, kas būs jāizmanto:
1. Planka konstante $h$
2. Gaismas ātrums $c$
3. Bolcmaņa konstante $k_B$

In [2]:
h=6.63e-34u"J*s"     # Planka konstante
c=2.99792e8u"m/s"    # gaismas ātrums 
k_B=1.381e-23u"J/K"  # Bolcmaņa konstante

1.381e-23 J K^-1

## Funkcijas definēšana ##
Nākamais solis ir definēt funkciju, kas rēķinās absolūti melnā ķermeņa emisijas intensitāti pie dotas temperatūras un viļņa garuma. 

_Julia_ valodā definēsim funkciju šādi:

`function manaFukcija(mainigais1,mainigais2,...)
  izteiksmes
end`

Principā, funkcija dod atpakaļ pēdējo objektu, kas tika rēķinātā funkcijā. To var precizēt ar pavēli `return mansRezultats`

In [3]:
# Definēsim funkciju B. 
# Funkcijai dod viļņa garumu λ un melņā ķermeņa temperatūru T
# Funkcija did atpakaļ meļņa ķermeņa izstarotu intensitāti pie viļņa garumu λ 
function B(λ,T)
    return 2*(h*c^2/λ^5)*1/(exp(h*c/(λ*k_B*T))-1)
end

B (generic function with 1 method)

## Funkcijas tests ##
Tagad testēsim funkciju, dodot funkcijai konkrētas mainīgo vērtības. 
Tā kā izmantojām paketi Unitful.jl, varam dod mērvienības un saņemt atbildi pareizās mērvienībās.

In [4]:
B(500e-9u"m",5000u"K")  # Melna ķermeņa spīdība pie 500 nm un 5000 K

1.2090769961788389e13 J m^-3 s^-1

## Spektra integrēšana ##
Ir labi zināt intensitāti pie konkrētās frekvences, bet parasti ir jāzina kopīgā jauda, kura tiek izstarota pa kādu diapazonu. Lai to zinātu, ir jāintegrē spektrs. Mēs izmantosim paketi QuadGK.jl un funkciju `quadgk()`. Tā funkcija ļauj integrēt funkcija $f(x)$ vienā dimensijā šādi:

`integrāls, kļūda = quadgk(x->f(x), no, līdz, pielaujama kļūda)`

Šeit rezultātu dod, kā _Tuple_ ar diviem elementiem. 


In [5]:
# Izmantojot funkciju quadgk, skaitliski integrēsim funciju π*B(λ,T) pie T=5000K no λ=0 līdz λ=10 μm.
Radiance, err=quadgk(λ->pi*B(λ,5000u"K"), 0u"m", 10e-6u"m", rtol=1e-9)

(3.537410022471607e7 J m^-2 s^-1, 0.01843443090092478 J m^-2 s^-1)

## Zīmēt spektru ##
Lai zimētu spektru, mums ir jādefinē kādu diapazonu `λ_range=sākums:solis:beigas` pa kuram vēlāmies zīmēt.
Jādomā par soli. Liels solis dod lielu precizitāti, bet rēķinās lēni. 

Objektam `λ_range` ir daudz elementi. Funkcija `B(λ,T)` gaida tikai divus skaitļus - viļņa garumu un temperatūru -
bet rakstot `B.(λ_range,T)`, funkcija tiek izvērtēta atsevišķi pie katra `λ_range` elementa. Attiecīgi rezultāts būs
objekts are daudziem elementiem. 

Zīmēsim ar citu funkciju, `plot`. Pirmie elementi, ko dodām `plot` ir objekts ar x-vertībam `λ_range` un objekts ar 
y-vertībām `Bvals`. Pēc tam var dod parametrus. Pēc tam, saucot funkcija `plot!()` ar "!" updeito esošo grafiku.

In [6]:
# Tagad taisīsim vienkāršo grafiku
λ_range=0.1e-6u"m":0.01e-6u"m":5.0e-6u"m"   # definēsim λ diapazonu (sākums:solis:beigas)
T=5000u"K"                                  # definēsim temperatūru
# Lai pielietotu funkciju uz vektoru (λ_range) jāizmanto ".", lai pielietotu "pa elementiem"
Bvals=B.(λ_range,T);  # semikols slep rezultātus. Citādi mums varētu būt garš saraksts ar rezultātiem.
Radiance, err=quadgk(λ->pi*B(λ,T), 0u"m", 10e-6u"m", rtol=1e-9)  # Rēķināsim kopējo emisiju
println("Radiance = ", Radiance)   # Drukāsim
plot(λ_range, Bvals,xlabel="Viļņa garums", ylabel="Emisijas jauda",label=T)
myText=string("Integrālā emisija = ", string(Radiance))    # Veidot tekstu ar rezultātu
annotate!(λ_range[1],Bvals[1],(myText,:red,:left))         # Pievienot tekstu ("!" nozīmē "updeito esošo objektu")
title!("Melna ķermeņa emisijas spektrs")                   # Pievienot virsrakstu ("!" nozīmē "updeito esošo objektu")


Radiance = 3.537410022471607e7 J m^-2 s^-1


## Interaktīva grafika ##
Tagad veidosim interaktīvu grafiku izmantojot makro `@manipulate`. Tas strādā šādi:

`@manipulate for mansMainigais = minimums:solis:maksimums
    Kaut kādas darbības
end`

In [7]:
@manipulate for T=200u"K":50u"K":10000u"K"
    λ_range=0.1e-6u"m":0.01e-6u"m":5.0e-6u"m"
    Bvals=B.(λ_range,T);  # semikols slep rezultātus. Citādi mums varētu būt garš saraksts ar rezultātiem.
    Radiance, err=quadgk(λ->pi*B(λ,T), 0u"m", 10e-6u"m", rtol=1e-9)
    plot(λ_range, Bvals, xlabel="Viļņa garums", ylabel="Emisijas jauda",label=T)
    annotate!(λ_range[1],Bvals[1],(myText,:red,:left))
    title!("Melna ķermeņa emisijas spektrs")
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["T"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 197, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
99, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\User\\.julia\\packages\\Knockout\\3gjS1\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\User\\.julia\\packages\\Knockout\\3gjS1\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\User\\.julia\\packages\\InteractBase\\h8Fwe\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\User\\.julia\\packages\\InteractBase\\h8Fwe\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\User\\.julia\\packages\\Interact\\pVyN2\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000007d00e4e0, Task (runnable) @0x000000007d00e4e0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"200 K\",\"250 K\",\"300 K\",\"350 K\",\"400 K\",\"450 K\",\"500 K\",\"550 K\",\"600 K\",\"650 K\",\"700 K\",\"750 K\",\"800 K\",\"850 K\",\"900 K\",\"950 K\",\"1000 K\",\"1050 K\",\"1100 K\",\"1150 K\",\"1200 K\",\"1250

In [8]:
#plot(λ_range,Bvals)

